<a href="https://colab.research.google.com/github/leonarddocchagas/data_science/blob/master/recife_bom_de_bola.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img width='30%' align='Center' src='https://github.com/leonarddocchagas/data_science/blob/master/projetos/recife_bom_de_bola/logo_recife_bom_de_bola_2014.png?raw=true' />

Maior campeonato de futebol de campos de várzea do mundo, o Recife Bom de Bola promove a inclusão social e a integração entre as diversas comunidades do Recife através do Esporte. Realizado pela Prefeitura do Recife, por meio da Secretaria de Esportes do Recife (SESP), o torneio envolve no ano de 2021, cinco categorias e mais de 350 times.